In [1]:
// hacky way to set java path since macos breaks this
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
// note: make sim link between bounder and Bounder due to macos case sensitivity before hand
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

import $ivy.`com.lihaoyi::scalatags:0.12.0`

import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._

userhome: String = "/home/s"
jniPath: String = "/home/s/software/z3/build"
newPath: Array[String] = Array(
  "/home/s/software/z3/build",
  "",
  "/home/s/software/z3/build",
  "/home/s/software/z3/build",
  "/usr/java/packages/lib/amd64",
  "/usr/lib/x86_64-linux-gnu/jni",
  "/lib/x86_64-linux-gnu",
  "/usr/lib/x86_64-linux-gnu",
  "/usr/lib/jni",
  "/lib",
  "/usr/lib"
)
res0_3: String = ":/home/s/software/z3/build:/home/s/software/z3/build:/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib"
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/home/s/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot

In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.ExpTag
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.SpecSpace
import edu.colorado.plv.bounder.lifestate.SpecSignatures
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData
Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
var android_home = android_home_possible.find(p => File(p).exists()).get             
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

java.library.path set to: /home/s/software/z3/build::/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.ExpTag

import edu.colorado.plv.bounder.Action

import edu.colorado.plv.bounder.RunConfig

import edu.colorado.plv.bounder.PickleSpec

import edu.colorado.plv.bounder.lifestate.SpecSpace

import edu.colorado.plv.bounder.lifestate.SpecSignatures

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await

import almond.interpreter.api.DisplayData

android_home_possible: List[String] = List(
  "/home/s/Library/Android/sdk",
  "/home/s/Android/Sdk"
)
android_home: String = "/home/s/Android/Sdk"

In [3]:
val db = new ExperimentsDb(Some(s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}

Displayers.register(classOf[Seq[TableAble]], (people: Seq[TableAble]) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(people.head.headers.map(v => th(v))),
        for (row <- people) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
          }
        )
      ).render
    }
  ).asJava
})

Initializing database


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@44dbeeda
defined class Count
defined class StrRes
getCountResult: AnyRef with GetResult[Count] = <function1>
getStrResult: AnyRef with GetResult[StrRes] = <function1>
defined trait TableAble

In [6]:
case class JoinedResultRow(tag:String, summary:String, loc:List[String], qry:String, config:String, apk:String) extends TableAble{
    override def toString = s"ResultRow(summary = ${summary}, apk = ${apk}"
    def getSummary:BounderUtil.ResultSummary = read[BounderUtil.ResultSummary](summary)
    def headers = List("tag", "summary", "loc", "qry", "apk")
    def values = List (tag  ,  summary ,  loc , qry  , apk)
    def merge(other:JoinedResultRow):JoinedResultRow = {
        assert(tag == other.tag)
        assert(qry == other.qry)
        assert(config == other.config)
        assert(apk == other.apk)
        
        JoinedResultRow(tag, write[BounderUtil.ResultSummary](Driver.reduceResults(getSummary, other.getSummary)), loc ++ other.loc,qry, config, apk)
    }
}

implicit val getCoffeeResult = GetResult(r => JoinedResultRow(r.<<, r.<<, List(r.<<), r.<<, r.<<, r.<<))
object ReadResults{
//where cast(results.result::json->'summary' as varchar) like '%Witnessed%' or cast(results.result::json->'summary' as varchar) like '%timeout%'
    def selectResults() = {
        val q = sql"""select results.jobtag, cast(results.result::json->'summary' as varchar), results.loc, results.qry, jobs.config as cfg, cast(jobs.config::json->'apkPath' as varchar) as benchmark from results inner join jobs on jobs.id=results.jobid"""
        Await.result(db.db.run(q.as[JoinedResultRow]), 30 seconds)   
    }
    val allResults = selectResults()
    
    // Top model
    val allTopResults = allResults.filter(res => read[ExpTag](res.tag).specRefinement == "")
    // val topResultsByQuery = allTopResults.groupBy(res => (res.apk,res.qry)).map{
    //     case (_, rows) => rows.reduce((a:JoinedResultRow,b:JoinedResultRow) => a.merge(b))
    // }
    val allSummaries = allTopResults.map{res => res.summary}.toSet
}
println(s"all results: ${ReadResults.allResults.size}")
println(s"all top results: ${ReadResults.allTopResults.size}")
//println(s"all top results by qry: ${ReadResults.topResultsByQuery.size}")
ReadResults.allSummaries

all results: 1148
all top results: 1148


defined class JoinedResultRow
getCoffeeResult: AnyRef with GetResult[JoinedResultRow] = <function1>
defined object ReadResults
res5_5: Set[String] = HashSet(
  "\"\\\"IUnknown, no more axioms, failure.\\\"\"",
  "\"\\\"Ino active body present for method <jwtc.chess.JNI: int getNumBoard()>\\\"\"",
  "\"\\\"Unreachable\\\"\"",
  "\"\\\"Ijava.lang.RuntimeException: no active body present for method <org.liberty.android.fantastischmemo.ui.StatisticsScreen$ChartTask: com.github.mikephil.charting.charts.Chart generateChart(java.lang.Object)>\\\"\"",
  "\"\\\"Ijava.lang.RuntimeException: no active body present for method <org.navitproject.navit.NavitAddressSearchActivity: void callbackCancelAddressSearch(long)>\\\"\"",
  "\"\\\"Ino active body present for method <org.sipdroid.codecs.BV16: int encode(short[],int,byte[],int)>\\\"\"",
  "\"\\\"Ino active body present for method <org.navitproject.navit.NavitAddressSearchActivity: void callbackCancelAddressSearch(long)>\\\"\"",
  "\"\\\"Ino active

In [ ]:


case class Person(name: String, age: Int) extends TableAble{
    override def headers = List("Name","age")
    override def values = List(name,age)
}

val people = 
  Seq(
    Person("alice", 29),
    Person("bob", 23),
    Person("charlie", 34)
  )


// Note that as of now this approach doesn't seem to work with parameterized types like Seq[Person]
// which is probably due to type erasure.


people

